In [1]:
!pip install pandas
!pip install numpy
!pip install openmeteo-requests
!pip install requests-cache
!pip install retry-requests
!pip install timezonefinder

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/sbu/academics/cse-519-dsf/project/to-collect
%ls

In [2]:
import openmeteo_requests
import requests_cache
import pandas as pd
import numpy as np
from retry_requests import retry
import csv
import time
import datetime
from timezonefinder import TimezoneFinder

In [3]:
delay_features = [
    'DayOfWeek',
    'FlightDate',
    'IATA_CODE_Reporting_Airline',
    'Tail_Number',
    'Flight_Number_Reporting_Airline',
    'OriginAirportID',
    'Origin',
    'OriginState',
    'DestAirportID',
    'Dest',
    'DestState',
    'CRSDepTime',
    'DepTime',
    'DepDelayMinutes',
    'CRSArrTime',
    'ArrTime',
    'ArrDelayMinutes',
    'CRSElapsedTime',
    'ActualElapsedTime',
    'AirTime',
    'Cancelled',
    'CancellationCode',
    'Diverted',
    'CarrierDelay',
    'WeatherDelay',
    'NASDelay',
    'SecurityDelay',
    'LateAircraftDelay'
]

dtypes = {
    'DayOfWeek':                       int,
    'FlightDate':                      'string',
    'IATA_CODE_Reporting_Airline':     'string',
    'Tail_Number':                     'string',
    'Flight_Number_Reporting_Airline': int,
    'OriginAirportID':                 int,
    'Origin':                          'string',
    'OriginState':                     'string',
    'DestAirportID':                   int,
    'Dest':                            'string',
    'DestState':                       'string',
    'CRSDepTime':                      int,
    'DepTime':                         int,
    'DepDelayMinutes':                 int,
    'CRSArrTime':                      int,
    'ArrTime':                         int,
    'ArrDelayMinutes':                 int,
    'CRSElapsedTime':                  int,
    'ActualElapsedTime':               int,
    'AirTime':                         int,
    'Cancelled':                       int,
    'CancellationCode':                'string',
    'Diverted':                        int,
    'CarrierDelay':                    int,
    'WeatherDelay':                    int,
    'NASDelay':                        int,
    'SecurityDelay':                   int,
    'LateAircraftDelay':               int
}

In [4]:
airportCoordMap = {}

with open('latitude_longitude.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            print(f'Column names are {", ".join(row)}')
            line_count += 1
        else:
            airportCoordMap[row[0]] = (float(row[1]), float(row[2]))
            line_count += 1
    print(f'Processed {line_count} lines.')

print(airportCoordMap)

Column names are iata_code, latitude_deg, longitude_deg, elevation_m
Processed 366 lines.
{'JFK': (40.639447, -73.779317), 'LAX': (33.942501, -118.407997), 'DFW': (32.896801, -97.038002), 'OKC': (35.393101, -97.6007), 'OGG': (20.898543, -156.431212), 'HNL': (21.32062, -157.924228), 'SFO': (37.61899948, -122.375), 'MIA': (25.79319954, -80.29060364), 'ORD': (41.9786, -87.9048), 'IAH': (29.9843998, -95.34140015), 'BOS': (42.3643, -71.005203), 'DTW': (42.21239853, -83.35340118), 'SEA': (47.449162, -122.311134), 'MSP': (44.882, -93.221802), 'STL': (38.748697, -90.370003), 'MCO': (28.42939949, -81.30899811), 'KOA': (19.738783, -156.045603), 'LAS': (36.083361, -115.151817), 'MEM': (35.04240036, -89.97669983), 'PDX': (45.58869934, -122.5979996), 'DCA': (38.8521, -77.037697), 'SAN': (32.73360062, -117.1900024), 'TUS': (32.115004, -110.938053), 'CLT': (35.2140007, -80.94309998), 'SJC': (37.362452, -121.929188), 'PHX': (33.435302, -112.005905), 'SNA': (33.675701, -117.867996), 'LGA': (40.777199, 

In [5]:
def splitDate(ts):
    day = ts.strftime('%Y-%m-%d')
    hour = ts.hour
    return pd.Series([day, hour])

In [6]:
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)
url = "https://archive-api.open-meteo.com/v1/archive"
chunk_size = 150

def fetchWeatherData(airports, year, startMonth, endMonth):
    weather_df = pd.DataFrame()
    latitudes = [airportCoordMap[airport][0] for airport in airports]
    longitudes = [airportCoordMap[airport][1] for airport in airports]

    start_date = datetime.date(year, startMonth, 1)
    end_date = datetime.date(year, endMonth + 1, 1) - datetime.timedelta(days=1)  # go one month above and subtract 1 day

    # go seven days before start_date
    start_date = start_date - datetime.timedelta(days=7)
    # go seven days after end_date
    end_date = end_date + datetime.timedelta(days=7)

    # lats, longs = zip(*coords)
    for i in range(0, len(airports), chunk_size):
        lat_chunk = latitudes[i:i+chunk_size]
        long_chunk = longitudes[i:i+chunk_size]

        # Make sure all required weather variables are listed here
        # The order of variables in hourly or daily is important to assign them correctly below
        params = {
            "latitude": lat_chunk,
            "longitude": long_chunk,
            "start_date": str(start_date),
            "end_date": str(end_date),
            "hourly": [
                "temperature_2m",
                "relative_humidity_2m",
                "dew_point_2m",
                "apparent_temperature",
                "precipitation",
                "rain",
                "snowfall",
                "snow_depth",
                "weather_code",
                "surface_pressure",
                "cloud_cover",
                "cloud_cover_low",
                "cloud_cover_mid",
                "cloud_cover_high",
                "wind_speed_10m",
                "wind_speed_100m",
                "wind_gusts_10m",
                "shortwave_radiation",
                "direct_radiation",
                "diffuse_radiation",
                "direct_normal_irradiance",
                "terrestrial_radiation"
            ],
            "temperature_unit": "fahrenheit",
            "wind_speed_unit": "mph",
            "precipitation_unit": "inch",
            "timeformat": "unixtime",
            "timezone": "auto"
        }
        time.sleep(1)
        responses = openmeteo.weather_api(url, params=params)

        # Process first location. Add a for-loop for multiple locations or weather models

        j = i
        for response in responses:
            # response = responses[0]
            # print("Airport: ", airports[i])
            # print(f"Coordinates {response.Latitude()}°E {response.Longitude()}°N")
            # print(f"Elevation {response.Elevation()} m asl")
            # print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
            # print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")
    
            # Process hourly data. The order of variables needs to be the same as requested.
            hourly = response.Hourly()
            hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
            hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()
            hourly_dew_point_2m = hourly.Variables(2).ValuesAsNumpy()
            hourly_apparent_temperature = hourly.Variables(3).ValuesAsNumpy()
            hourly_precipitation = hourly.Variables(4).ValuesAsNumpy()
            hourly_rain = hourly.Variables(5).ValuesAsNumpy()
            hourly_snowfall = hourly.Variables(6).ValuesAsNumpy()
            hourly_snow_depth = hourly.Variables(7).ValuesAsNumpy()
            hourly_weather_code = hourly.Variables(8).ValuesAsNumpy()
            hourly_surface_pressure = hourly.Variables(9).ValuesAsNumpy()
            hourly_cloud_cover = hourly.Variables(10).ValuesAsNumpy()
            hourly_cloud_cover_low = hourly.Variables(11).ValuesAsNumpy()
            hourly_cloud_cover_mid = hourly.Variables(12).ValuesAsNumpy()
            hourly_cloud_cover_high = hourly.Variables(13).ValuesAsNumpy()
            hourly_wind_speed_10m = hourly.Variables(14).ValuesAsNumpy()
            hourly_wind_speed_100m = hourly.Variables(15).ValuesAsNumpy()
            hourly_wind_gusts_10m = hourly.Variables(16).ValuesAsNumpy()
            hourly_shortwave_radiation = hourly.Variables(17).ValuesAsNumpy()
            hourly_direct_radiation = hourly.Variables(18).ValuesAsNumpy()
            hourly_diffuse_radiation = hourly.Variables(19).ValuesAsNumpy()
            hourly_direct_normal_irradiance = hourly.Variables(20).ValuesAsNumpy()
            hourly_terrestrial_radiation = hourly.Variables(21).ValuesAsNumpy()
    
            hourly_data = {"date": pd.date_range(
                start = pd.to_datetime(hourly.Time(), unit = "s"),
                end = pd.to_datetime(hourly.TimeEnd(), unit = "s"),
                freq = pd.Timedelta(seconds = hourly.Interval()),
                inclusive = "left"
            )}
    
            hourly_data["temperature_2m"] = hourly_temperature_2m
            hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
            hourly_data["dew_point_2m"] = hourly_dew_point_2m
            hourly_data["apparent_temperature"] = hourly_apparent_temperature
            hourly_data["precipitation"] = hourly_precipitation
            hourly_data["rain"] = hourly_rain
            hourly_data["snowfall"] = hourly_snowfall
            hourly_data["snow_depth"] = hourly_snow_depth
            hourly_data["weather_code"] = hourly_weather_code
            hourly_data["surface_pressure"] = hourly_surface_pressure
            hourly_data["cloud_cover"] = hourly_cloud_cover
            hourly_data["cloud_cover_low"] = hourly_cloud_cover_low
            hourly_data["cloud_cover_mid"] = hourly_cloud_cover_mid
            hourly_data["cloud_cover_high"] = hourly_cloud_cover_high
            hourly_data["wind_speed_10m"] = hourly_wind_speed_10m
            hourly_data["wind_speed_100m"] = hourly_wind_speed_100m
            hourly_data["wind_gusts_10m"] = hourly_wind_gusts_10m
            hourly_data["shortwave_radiation"] = hourly_shortwave_radiation
            hourly_data["direct_radiation"] = hourly_direct_radiation
            hourly_data["diffuse_radiation"] = hourly_diffuse_radiation
            hourly_data["direct_normal_irradiance"] = hourly_direct_normal_irradiance
            hourly_data["terrestrial_radiation"] = hourly_terrestrial_radiation
            hourly_data['airport'] = airports[j]
    
            hourly_dataframe = pd.DataFrame(data = hourly_data)

            hourly_dataframe[['day', 'hour']] = hourly_dataframe['date'].apply(splitDate)
            hourly_dataframe.drop('date', axis=1, inplace=True)

            weather_df = pd.concat([weather_df, hourly_dataframe], sort=False, ignore_index=True)
            j += 1

    print(weather_df)
    return weather_df

In [9]:
airports_to_remove = ['USA']
tf = TimezoneFinder()


def get_arrival_time(row):
    dep_date = datetime.datetime.strptime(row['FlightDate'], "%Y-%m-%d").date()
    temp_time = datetime.datetime.strptime(row['CRSDepTime'], '%H%M').replace(tzinfo=pytz.timezone(row['OriginTz'])).time()
    dep_time = datetime.combine(dep_date, temp_time)

    flight_time = datetime.timedelta(minutes=row['CRSElapsedTime'])
    dep_delay_minutes = datetime.timedelta(minutes=row['DepDelayMinutes'])

    arrival_time = dep_time + dep_delay_minutes + flight_time
    arrival_time = arrival_time.astimezone(tz=pytz.timezone(row['DestTz']))
    
    return arrival_time


def mergeDelayAndWeather(year, startMonth, endMonth):
    # read csvs of all the months
    months = [f'{n:02}' for n in range(startMonth, endMonth + 1)]
    df_all_months = (pd.read_csv(f'data/airline-performance/individual/aperf-{year}-{month}.csv', usecols=delay_features, dtype=dtypes) for month in months)
    delay_df = pd.concat(df_all_months, ignore_index=True)

    print(delay_df[delay_df["CRSElapsedTime"].isna()]["Cancelled"])

    # drop irrelevant records
    # delay_df = delay_df[(delay_df['WeatherDelay'] > 0) & (delay_df['NASDelay'] > 0)]

    # find all unique airports
    origin_airports = delay_df['Origin'].unique().tolist()
    dest_airports = delay_df['Dest'].unique().tolist()
    airports = list(set(origin_airports + dest_airports))
    
    # remove unnecessary airports
    airports = [x for x in airports if x not in airports_to_remove]
    missing_airports = [airport for airport in airports if airport not in airportCoordMap]

    print("Number of airports: ", len(airports))
    if len(missing_airports) > 0:
        print("Missing airports: ", missing_airports)

    # compute departure
    delay_df['CRSDepHour'] = delay_df['CRSDepTime'].apply(lambda x: max(0, (x - 15)//100))
    delay_df['OriginTz'] = delay_df['Origin'].apply(lambda x: tf.timezone_at(lng=airportCoordMap[x][1], lat=airportCoordMap[x][0]))

    # compute arrival
    delay_df['DestTz'] = delay_df['Dest'].apply(lambda x: tf.timezone_at(lng=airportCoordMap[x][1], lat=airportCoordMap[x][0]))
    delay_df['CRSArrDateTime'] = delay_df.apply(get_arrival_time, axis=1)
    delay_df['CRSArrDate'] = delay_df['CRSArrDateTime'].apply(lambda x: x.strftime("%Y-%m-%d"))
    delay_df['CRSArrHour'] = delay_df['CRSArrDateTime'].apply(lambda x: max(0, ((x.hour * 100 + x.minute) - 15)//100))

    # get weather data
    weather_df = fetchWeatherData(airports, year, startMonth, endMonth)
    # print("Delay data:\n", delay_df.describe())
    # print("Weather data:\n", weather_df.describe())
    
    merged_data = pd.merge(delay_df, weather_df, left_on=['Origin', 'FlightDate', 'CRSDepHour'], right_on=['airport', 'day', 'hour'], how='left')
    merged_data = pd.merge(merged_data, weather_df, left_on=['Dest', 'CRSArrDate', 'CRSArrHour'], right_on=['airport', 'day', 'hour'], how='left')

    print("Merged data", merged_data.describe())
    return merged_data

In [10]:
for year in [2017, 2018, 2019]:
    data = mergeDelayAndWeather(year, 1, 1)
    data.to_csv(f'data/weather-joined/full-w-{year}-01.csv', index=False)
    break

4
Number of airports:  298


KeyboardInterrupt: 